In [ ]:
# 初始化
import django_setup

In [2]:
from apps.doc_analysis.LLM_services._generic_llm_services import GenericLLMService
from apps.doc_analysis.LLM_services._llm_types import LLMConfig, LLMRequest, AnalysisOutput
import asyncio, os, nest_asyncio
nest_asyncio.apply()

In [3]:
config = LLMConfig(
    llm_model_name = "qwen-plus",
    temperature = 0.7,
    top_p =  0.8,
    streaming = True,
    api_key = os.getenv("ALIBABA_API_KEY"),
    base_url = "https://dashscope.aliyuncs.com/compatible-mode/v1",
    max_workers = 4,
    timeout = 30,
    retry_times = 3
)

In [4]:
# 2. 创建服务实例
service = GenericLLMService(config)

In [5]:
# 3. 设置prompt模板
prompt_template = """
请根据以下内容回答问题：
{context}

问题：{requirement}
"""
service.prompt_template = prompt_template

In [6]:
# 4. 准备测试数据
context = """
人工智能（AI）是计算机科学的一个分支，它企图了解智能的实质，并生产出一种新的能以人类智能相似的方式做出反应的智能机器。
"""
requirement = "请简要解释什么是人工智能"

In [7]:
# 5. 创建请求对象
request = LLMRequest(context=context, requirement=requirement)

In [8]:
# 6. 异步调用服务
async def test_service():
    try:
        result = await service.process(request, stream=True)
        print("处理结果：", result)
    except Exception as e:
        print("发生错误：", str(e))

In [ ]:
# 7. 运行测试
await test_service()

测试_02_example_analysis

In [1]:
# 初始化
import django_setup

Development settings loaded
INSTALLED_APPS: ['django.contrib.admin', 'django.contrib.auth', 'django.contrib.contenttypes', 'django.contrib.sessions', 'django.contrib.messages', 'django.contrib.staticfiles', 'rest_framework', 'corsheaders', 'storages', 'apps.authentication', 'apps.files', 'apps.projects', 'apps.doc_analysis', 'apps.chat', 'django_filters', 'drf_spectacular', 'rest_framework_simplejwt.token_blacklist', 'django_celery_results', 'django_celery_beat']


INFO 2025-02-21 02:31:05,753 storage default_storage 的类型: COSStorage


Settings从哪里加载？: config.settings.development
项目根目录对么？: C:\Users\huiwa\Documents\_All_Projects\BidPilot_new\backend
文件存储settings对么？: apps.files.storage.COSStorage
文件default_storage对么？: COSStorage

已经安装的应用 Installed Apps 完整了么？:
- django.contrib.admin
- django.contrib.auth
- django.contrib.contenttypes
- django.contrib.sessions
- django.contrib.messages
- django.contrib.staticfiles
- rest_framework
- corsheaders
- storages
- apps.authentication
- apps.files
- apps.projects
- apps.doc_analysis
- apps.chat
- django_filters
- drf_spectacular
- rest_framework_simplejwt.token_blacklist
- django_celery_results
- django_celery_beat


In [2]:
# 导入相关模型：get_user_model, Project, FileRecord, DocumentAnalysis, FileProjectLink, ProjectHistory
from apps.doc_analysis.models import DocumentAnalysis
from apps.doc_analysis.pipeline.types import DocxElements, OutlineAnalysisResult
from pprint import pprint 
import os
# call llm service
from apps.doc_analysis.LLM_services._01_example import OutlineLLMAnalysis
from apps.doc_analysis.LLM_services._generic_llm_services import GenericLLMService, LLMRequest
from apps.doc_analysis.LLM_services._batch_llm_services import BatchLLMService

In [4]:
# 直接引用"测试分析A", 并获取其extracted_elements

saved_analysis = DocumentAnalysis.objects.get(title="测试分析A")
extracted_elements = saved_analysis.extracted_elements
pprint(extracted_elements['elements'][10])

{'content': '前附表',
 'is_toc': True,
 'position': 11,
 'toc_level': 2,
 'type': 'paragraph'}


In [5]:
# 创建 DocxElements 实例 from models.py 的 extracted_elements
docx_elements = DocxElements.from_model(extracted_elements)



In [6]:
# build context,  参数 context
formatted_toc = docx_elements.format_toc_chapters()
formatted_headings = docx_elements.format_heading_chapters()
def _build_context(formatted_toc: str, formatted_headings: str) -> str:
    return f"""
1. 目录标题列表：从文档目录中提取的标题
2. 正文标题列表：从文档正文中提取的标题

数据格式：
"[文档位置], 标题层级, 标题内容"

数据内容：
1. <目录标题列表>：
{formatted_toc}

2. <正文标题列表>：
{formatted_headings}
"""

context = _build_context(formatted_toc, formatted_headings)
print(context)


1. 目录标题列表：从文档目录中提取的标题
2. 正文标题列表：从文档正文中提取的标题

数据格式：
"[文档位置], 标题层级, 标题内容"

数据内容：
1. <目录标题列表>：
[8], Lvl:1, 标题：第一章  招标公告
[9], Lvl:1, 标题：第二章  招标需求
[10], Lvl:1, 标题：第三章  投标人须知
[21], Lvl:1, 标题：第四章  评标办法及评分标准
[25], Lvl:1, 标题：第五章  合同条款及格式
[26], Lvl:1, 标题：第六章  投标文件格式

2. <正文标题列表>：
[1], Lvl:1, 标题：北京京铁运恒采购供应站有限公司 2024 年端午节物资采购项目
[2], Lvl:1, 标题：招标文件
[46], Lvl:1, 标题：第一章  招标公告
[95], Lvl:1, 标题：第二章 招标需求
[124], Lvl:1, 标题：第三章  投标人须知
[308], Lvl:1, 标题：第四章  评标办法及评分标准
[366], Lvl:1, 标题：第五章 合同条款及格式
[501], Lvl:1, 标题：第六章  投标文件格式



In [7]:
# build requirement， 参数 requirement
def _build_requirement() -> str:
    """
    构建大模型分析任务要求
    """
    return """
请对比<目录标题列表>和<正文标题列表>的标题内容
找出以下三类不同标题项：
1. 目录列表里有，但正文里没有的标题项
2. 目录列表里没有，但正文里有的标题项

请注意：只比对标题内容，不比对[文档位置] 和 标题层级
"""

requirement = _build_requirement()
print(requirement)



请对比<目录标题列表>和<正文标题列表>的标题内容
找出以下三类不同标题项：
1. 目录列表里有，但正文里没有的标题项
2. 目录列表里没有，但正文里有的标题项

请注意：只比对标题内容，不比对[文档位置] 和 标题层级



In [8]:
# build output_format, 参数 output_format
def _build_output_format() -> str:
    """
    构建大模型分析的输出要求
    """
    return OutlineAnalysisResult.get_prompt_specification()

output_format = _build_output_format()
print(output_format)


请严格按照以下JSON格式输出目录分析结果，不要包含任何额外的解释或说明：
{
    # 目录独有的元素列表
    "toc_only_elements": [
        {
            "position": int,  # 元素在文档中的位置索引
            "content": str,   # 元素内容
            "is_toc": bool,   # 是否为目录
            "toc_level": int, # 目录级别（如1, 2, 3等）
            "reason": str,        # 判断为入选的原因
            "recommendation":str, # 建议在正文中找到对应元素，并将其改为与目录层级匹配的正文标题
            "confidence": float,  # 评估recommendation可信度，0.0(完全不可信)~1.0(完全确信) 
            "user_confirm": False,     # 尚未用户确认，所以一致取值False  
        }
    ],
    # 正文独有的元素列表
    "heading_only_elements": [
        {
            "position": int,      # 元素在文档中的位置索引
            "content": str,       # 元素内容
            "is_heading": bool,   # 是否为标题
            "heading_level": int, # 标题级别（如1, 2, 3等）
            "reason": str,        # 判断为入选的原因
            "recommendation":str, # 建议在正文中找到对应标题元素，取消标题格式
            "confidence": float,  # 评估recommendation可信度，0.0(完全不可信)~1.0(完全确信) 
            "user_confirm": False,     # 尚未用户确认，所

In [ ]:
result = await OutlineLLMAnalysis.analyze(context, requirement, output_format)

In [9]:
# 提取<目录标题列表>和<正文标题列表>, formatted的格式，而不是Json格式
# 1. 目录标题列表
toc_chapters = docx_elements.format_toc_chapters()
toc_sections = docx_elements.format_toc_sections()
toc_subsections = docx_elements.format_toc_subsections()

# 2. 正文标题列表
heading_chapters = docx_elements.format_heading_chapters()
heading_sections = docx_elements.format_heading_sections()
heading_subsections = docx_elements.format_heading_subsections()


# 3. contexts
chapter_context = OutlineLLMAnalysis.build_context(toc_chapters, heading_chapters)
section_context = OutlineLLMAnalysis.build_context(toc_sections, heading_sections)
subsection_context = OutlineLLMAnalysis.build_context(toc_subsections, heading_subsections)

# 创建多个分析请求
requests = [
    LLMRequest(context=chapter_context, requirement=requirement, output_format=output_format),
    LLMRequest(context=section_context, requirement=requirement, output_format=output_format),
    LLMRequest(context=subsection_context, requirement=requirement, output_format=output_format),
    ]

In [10]:
batch_service = BatchLLMService()

In [11]:
result = await batch_service.process(requests)


输入参数错误: Prompt template is required


ValueError: Prompt template is required